In [5]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import glob, sys
import sys
import xarray as xr
import dask.distributed
from matplotlib import colors as c
import pytide
import multiprocessing
import pandas as pd
import datetime
import seawater

Read variable in a netcdf file. Function requires variable name, xaxis,yaxis and zaxis grid name.
mask file is also creating. Now the value greater than 2000 is used to create mask file. This value can set according to the model output. 2000 is given to consider density value which range 1000.
Timeseries values is converting to datetime64 format which is required for harmonic analysis in pytide package

In [7]:
def read_netcdf(variable_name,t_name,x_name,y_name,k_name):
    data_netcdf=xr.open_mfdataset('ROMS_*.nc', parallel=True, concat_dim=t_name, combine="nested",
                           data_vars='minimal', coords='minimal', compat='override')
    if variable_name == 'ZETA':
        data_netcdf_var=data_netcdf[variable_name][:,:,:].values
        mask=xr.where(data_netcdf_var[0,:,:]>100, np.nan, 1)
    else:
        data_netcdf_var=data_netcdf[variable_name][:,:,:,:].values
        mask=xr.where(data_netcdf_var[0,:,:,:]>100, np.nan, 1) 
    lon=data_netcdf[x_name][:].values
    lat=data_netcdf[y_name][:].values
    depth=data_netcdf[k_name][:].values
    time_value=data_netcdf[t_name].values.astype('datetime64[s]')
    
    xsize=lon.size
    ysize=lat.size
    zsize=depth.size
    if variable_name == 'ZETA':
        zsize=1
    return data_netcdf_var, mask, time_value,lon,lat,depth,xsize,ysize,zsize

In [8]:
def nodal_modulations(wave_table, time_series):
   # """Compute nodal corrections for a given time series"""    
    #t = time_series.astype('datetime64[s]')
    waves_table = pytide.WaveTable(wave_table)
    f, v0u = waves_table.compute_nodal_modulations(time_series)
    return (f,v0u)

In [9]:
def Harmonic_analysis(mask,eta,f,v0u,wave_table):
    #if mask !=0:
    if mask ==1:
        #print ("Passed")
        waves_table = pytide.WaveTable(wave_table)
        w = waves_table.harmonic_analysis(eta, f, v0u)
        return w 
    return complex(0)

In [10]:
def splat_harmonic_analysis(args):
    return Harmonic_analysis(*args)

In [11]:
def parallel(var,mask,xsize,ysize,zsize,eta,f,v0u,wave_table):
    pool=multiprocessing.Pool(4)
    #amp = pool.map(splat_harmonic_analysis,
    #        ((mask[i,j],eta[:,i,j], f, v0u,waves_table) for i in range(xsize) for j in range(ysize)))
    if var == 'ZETA':
        analysis = pool.map(splat_harmonic_analysis,
                   ((mask[i,j],eta[:,i,j], f, v0u,wave_table) for i in range(xsize) for j in range(ysize)))
    else:
        analysis = pool.map(splat_harmonic_analysis,
                   ((mask[k,i,j],eta[:,k,i,j], f, v0u,wave_table) for k in range(zsize) for i in range(xsize) 
                   for j in range(ysize)))
        
    return analysis

In [12]:
def create_netcdf(var,x,y,z,amp,pha):
    if var=='ZETA':
        z=[0.0]
        #amp=np.reshape(amp,[x.size,y.size,1])
        #pha=np.reshape(pha,[x.size,y.size,1])       
    ds = xr.Dataset(
        data_vars=dict(
            amp=(["depth","lon", "lat"], amp),
            pha=(["depth","lon", "lat"], pha)),
        coords=dict(
            lon=(["lon"], x),
            lat=(["lat"], y),
            depth=(["depth"], z),
        ),
        attrs=dict(description=var),
    )
    if var=='ZETA':
        ds.amp.attrs["units"] = "meters"
    elif var=='U' or var=='V':
        ds.amp.attrs["units"] = "m/s"
    else:
        ds.amp.attrs["units"] = "meters"            
    ds.pha.attrs["units"] = "degree"
    ds.lon.attrs["units"] = "degree_east"
    ds.lat.attrs["units"] = "degree_north"
    
    out_data=ds.to_netcdf(var+'.nc',"w")
    
    return out_data

In [13]:
def timeseries_constituents(wave_table,xsize,ysize,zsize,amp,pha):
    constituent=wave_table[0]
    if constituent=='M2':
        time=np.arange(13)
        tide =  np.reshape(amp,(1,zsize,xsize,ysize)) * np.sin((0.505636071* 
                                                     np.tile(time.reshape(13,1,1,1),
                                                                          (1,zsize,xsize,ysize)))- 
                                                    (np.reshape(pha,(1,zsize,xsize,ysize))*(np.pi/180.)))
    return tide

In [14]:
def create_netcdf_timeseries(wave_table,var,x,y,z,t,tide):
    constituent=wave_table[0]
    if constituent=='M2':
        t=t[0:13]
        #t=pd.to_datetime(time[0:13])
    if var=='ZETA':
        z=[0.0]    
    ds = xr.Dataset(
        data_vars=dict(
            tide=(["time","depth","lon", "lat"], tide),),
        coords=dict(
            lon=(["lon"], x),
            lat=(["lat"], y),
            depth=(["depth"], z),
            time=(["time"],t)
        ),
        attrs=dict(description=var),
    )
    if var=='ZETA':
        ds.tide.attrs["units"] = "meters"
    elif var=='U' or var=='V':
        ds.tide.attrs["units"] = "m/s"
    else:
        ds.tide.attrs["units"] = "meters"            
    ds.lon.attrs["units"] = "degree_east"
    ds.lat.attrs["units"] = "degree_north"
#    ds.time.attrs["units"] = "hours"
#    ds.time.attrs["calendar"] = "Julian"
    
    out_data=ds.to_netcdf(var+'_timeseries.nc',"w")
    
    return out_data

In [15]:
def vertical_average(var,depth,mask,xsize,ysize):
    dz=np.append([1],(depth[1:]-depth[:-1]))  # dz value will start with [1,...] 
    
    #  sum of ( variable* mask * dz) / sum of (dz*mask) 
    depth_mean= np.nansum((var*(mask.reshape(1,dz.size,xsize,ysize)))*
                       (dz.reshape(1,dz.size,1,1)),axis=1)/np.nansum( (
        (dz.reshape(1,dz.size,1,1))* (mask.reshape(1,dz.size,xsize,ysize)) ),axis=1 )
    
    #depth_mean2= np.sum((var)*(dz.reshape(1,dz.size,1,1)),axis=1)/np.sum(dz)
    return(depth_mean)

In [16]:
def bv_frequency(salt,temp,depth,mask):
    bv=seawater.bfrq(df.salt,df.temp,df.dep,lat=None)
    seawater.dens
    return bv,density

In [17]:
if __name__ == '__main__':
    
    ### User defined NETCDF variable and axis names
    var_name ='U'
    var_tname='TIME'
    var_xname='LON_RHO385_409'
    var_yname='LAT_RHO122_146'
    var_zname='DEPTH'
    
    ### User defined tidal consituent
    wave_table=['M2'] 
    
    ##
    #mask=1
    
    
    eta,mask,time,lon,lat,depth,xsize,ysize,zsize=read_netcdf(var_name,var_tname,var_xname,var_yname,var_zname)


    f,v0u=nodal_modulations(wave_table,time)
    
   
    analysis=parallel(var_name,mask,xsize,ysize,zsize,eta,f,v0u,wave_table)
    
    amp=np.absolute (np.reshape(np.array(analysis), (zsize,xsize,ysize)))
    
    pha=np.angle (np.reshape(np.array(analysis), (zsize,xsize,ysize)),deg=True)
    
    
    
    out=create_netcdf(var_name,lon,lat,depth,amp,pha)
    
    tide=timeseries_constituents(wave_table,xsize,ysize,zsize,amp,pha)

    create_netcdf_timeseries(wave_table,var_name,lon,lat,depth,time,tide)



In [19]:

    u_mean=vertical_average(tide,depth,mask,xsize,ysize)



In [21]:
##### Isopycnal displacement
print(u_mean.shape)


(13, 25, 25)
